In [1]:
!pip install ibm_watson bs4

     |████████████████████████████████| 389kB 9.3MB/s eta 0:00:01
     |████████████████████████████████| 204kB 8.6MB/s eta 0:00:01
     |████████████████████████████████| 122kB 8.4MB/s eta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/6e/14/69/dbbd573a3bab3bf64984572284f13f174f430038308abdd73c
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/a0/b0/b2/4f80b9456b87abedbc0bf2d52235414c3467d8889be38dd472
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/34/6e/58/589e0f841c2fae9dad99630d78ddc7a60c5c7663a16a39cdbb
Successfully built ibm-watson bs4 ibm-cloud-sdk-core


In [2]:
import os
from glob import glob
from bs4 import BeautifulSoup
import IPython
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
from ibm_watson import SpeechToTextV1
from ibm_watson import AssistantV2
from ibm_watson import TextToSpeechV1

In [3]:
recognition_service = SpeechToTextV1(IAMAuthenticator('B04gouRAgdg3kKswG22ZUGuHOg3rgPC8RhEeGRXC34V_'))
recognition_service.set_service_url('https://api.eu-gb.speech-to-text.watson.cloud.ibm.com/instances/82878fec-31d1-4207-8b53-1f01005ad7b1')
SPEECH_EXTENSION = "*.webm"
SPEECH_AUDIOTYPE = "audio/webm"

def recognize_audio():
    while len(glob(SPEECH_EXTENSION)) == 0:
        pass
    filename = glob(SPEECH_EXTENSION)[0]
    audio_file = open(filename, "rb")
    os.popen("rm " + filename)
    result = recognition_service.recognize(audio=audio_file, content_type=SPEECH_AUDIOTYPE).get_result()
    return result["results"][0]["alternatives"][0]["transcript"]

In [4]:
assistant = AssistantV2(version='2019-02-28', authenticator=IAMAuthenticator('M9gl7sZWa4hv9FpnWnTIqfWZuWqswRNQh-FBAwViewu6'))
assistant.set_service_url('https://api.eu-gb.assistant.watson.cloud.ibm.com/instances/c9ca729c-f9f4-4915-a392-a0497ea186a2')
ASSISTANT_ID = "fb4019de-f018-4de0-abb4-96533d2a7a31"
session_id = assistant.create_session(assistant_id=ASSISTANT_ID).get_result()["session_id"]

def message_assistant(text):
    response = assistant.message(assistant_id=ASSISTANT_ID,
                                 session_id=session_id,
                                 input={'message_type': 'text', 'text': text}).get_result()
    return BeautifulSoup(response["output"]["generic"][0]["text"]).get_text()

In [5]:
synthesis_service = TextToSpeechV1(IAMAuthenticator('1DzRFafPv8nCTIH8-odaWVJ2OcwwHY-izNQ8FICQxxsI'))
synthesis_service.set_service_url('https://api.eu-gb.text-to-speech.watson.cloud.ibm.com/instances/a7450141-a9cf-4269-aeab-6b8d6f344d8e')

def speak_text(text):
    with open('temp.wav', 'wb') as audio_file:
        response = synthesis_service.synthesize(text, accept='audio/wav', voice="en-US_AllisonV3Voice").get_result()
        audio_file.write(response.content)
    return IPython.display.Audio("temp.wav", autoplay=True)

In [ ]:
speak_text(message_assistant(recognize_audio()))